# Library Imports

In [1]:
import numpy as np
import pandas as pd

# Data Import checks

In [2]:
raw_drinkchem = pd.read_excel('./assets/Drinkingwater/chemical.xlsx')

In [4]:
print(raw_drinkchem.shape)
raw_drinkchem.head()

(1048575, 12)


,PRIM_STA_C,SAMP_DATE,SAMP_TIME,LAB_NUM,ANADATE,INDATE,METHOD,INBY,SPECIAL,STORE_NUM,XMOD,FINDING
0,0103039-004,2017-11-22,848,4790,2017-11-22,2017-11-30,SM 212,#,S,00081,<,3.0
1,0103039-004,2017-11-22,848,4790,2017-11-22,2017-11-30,SM 215,#,S,00086,<,1.0
2,0103039-004,2017-11-22,848,4790,2017-11-29,2017-11-30,SM 251,#,S,00095,NaN,550.0
3,0103039-004,2017-11-22,848,4790,2017-11-29,2017-11-30,SM4500,#,S,00403,NaN,8.1
4,0103039-004,2017-11-22,848,4790,2017-11-22,2017-11-30,300.0,#,S,00618,NaN,5.7


In [6]:
raw_drinksys = pd.read_excel('./assets/Drinkingwater/watsys.xlsx')

In [7]:
print(raw_drinksys.shape)
raw_drinksys.head()

(15056, 11)


,SYSTEM_NO,SYSTEM_NAM,HQNAME,ADDRESS,CITY,STATE,ZIP,ZIP_EXT,POP_SERV,CONNECTION,AREA_SERVE
0,2800064,DAKOTA SHY WINERY,NaN,771 Sage Canyon Road,ST. HELENA,CA,94574,NaN,30.0,1.0,NaN
1,5403214,NaN,THREE RIVERS HISTORICAL MUSEUM,42268 SIERRA DR,THREE RIVERS,CA,93271,NaN,25.0,2.0,NaN
2,4000829,VINES R.V. RESORT,NaN,88 WELLSONA,PASO RO,CA,93446,NaN,400.0,1.0,NaN
3,0103039,MOHRLAND MUTUAL WATER SYSTEM,MOHRLAND MUTUAL,24927 Mohr Drive,HAYWARD,CA,94545,tt,116.0,99.0,NaN
4,0103040,NORRIS CANYON PROPERTY OWNERS ASSN.,NORRIS CANYON P,8653 Norris Canyon Road,CASTRO VALLEY,CA,94552,S,50.0,19.0,NaN


In [8]:
raw_drinksite = pd.read_excel('./assets/Drinkingwater/siteloc.xlsx')

In [9]:
print(raw_drinksite.shape)
raw_drinksite.head()

(58328, 11)


,PRI_STA_C,FRDS_NO,COUNTY,DISTRICT,USER_ID,SYSTEM_NO,WATER_TYPE,SOURCE_NAM,STATION_TY,STATUS,COMMENT_1
0,4301014-001,4301014001,43,17,SAN,4301014,G,WELL 01 - TREATED,NaN,AT,NaN
1,4301014-002,4301014002,43,17,SAN,4301014,G,WELL 01- CL2 TREATMENT,NaN,DT,NaN
2,2300906-002,2300906002,23,3,RXR,2300906,G,DBP SAMPLE LOCATION,NaN,DT,NaN
3,2310006-030,2310006030,23,3,RXR,2310006,G,DBP 781 TOKAY DRIVE,NaN,DT,NaN
4,2300955-001,2300955001,23,3,RXR,2300955,G,WELL 01,NaN,AR,NaN
